In [1]:
# Dependencies for the MCP server
%pip install impyla python-dotenv "mcp[cli]" --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies for the application
%pip install langchain-mcp-adapters langgraph langchain-openai --quiet 

Note: you may need to restart the kernel to use updated packages.


In [3]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI

In [4]:
# Configuring AI Inference service hosted model Endpoint
import json

API_KEY = json.load(open("/tmp/jwt"))["access_token"]
MODEL_NAME = "meta/llama-3.1-8b-instruct"  # Update this for your Cloudera AI Inference service Endpoint
BASE_URL = "https://ml-2dad9e26-62f.cloudera.site/namespaces/serving-default/endpoints/llama3-8b-throughput/v1/"  # Update this for your Cloudera AI Inference service Endpoint

model = ChatOpenAI(model=MODEL_NAME, base_url=BASE_URL, api_key=API_KEY)

In [5]:
# Configuring MCP Server and Impala endpoint
server_params = StdioServerParameters(
    command="python",
    args=["/home/cdsw/iceberg-mcp-server/server.py"],
    env={
        "IMPALA_HOST": "coordinator-default-impala.example.cloudera.site",  # Update this for your Impala host
        "IMPALA_USER": "username",
        "IMPALA_PASSWORD": "password",
    },
)

In [6]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke(
            {
                "messages": "What information can I learn form the `airlines_snappy` table? I'm looking for a quick answer, don't ask for clarifying questions."
            }
        )

In [7]:
agent_response

{'messages': [HumanMessage(content="What information can I learn form the `airlines_snappy` table? I'm looking for a quick answer, don't ask for clarifying questions.", additional_kwargs={}, response_metadata={}, id='0512e26e-1f57-4a72-a621-3ba835d9183b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-eae3dcd9672c4ebd8591c2107765090e', 'function': {'arguments': '{"query": "SELECT * FROM airlines_snappy LIMIT 1"}', 'name': 'execute_query'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 290, 'total_tokens': 319, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta/llama-3.1-8b-instruct', 'system_fingerprint': None, 'id': 'chat-aeb8b919ea254ef49ba2be21633fa8d0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-051bdb8d-57a9-4c86-822a-9f59d195a4b3-0', tool_calls=[{'name': 'execute_query', 'args': {'query': 'SELECT * FROM airlines_snappy LIMIT 1

In [8]:
print(agent_response["messages"][3].content)

The `airlines_snappy` table contains information about airlines, including their ID, name, size, carriers, fights, destinations, seat_capacity, fleet_age, and service.
